In [38]:
import ast
from termcolor import cprint

In [39]:
def check_for_docstring(node):
  if hasattr(node, 'body') and node.body and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Str):
    return True
  return False

In [40]:
def walk_python_code(source_code_path):
  with open(source_code_path) as file:
    source_code = file.read()
  
  tree = ast.parse(source_code)

  for node in ast.walk(tree):
    # Skip nodes that already have a docstring
    if check_for_docstring(node):
      cprint(f'node {node} already has a docstring', 'red')
      cprint('-' * 80)
      continue
    
    if isinstance(node, (ast.FunctionDef, ast.AsyncFunctionDef)):
      cprint(f'func node name: {node.name}', 'yellow')
      cprint(f'func node docstring: {ast.get_docstring(node)}', 'yellow')
      cprint(f'func node source code: {ast.get_source_segment(source_code, node)}', 'yellow')
      cprint('-' * 80)
    elif isinstance(node, ast.ClassDef):
      cprint(f'class node name: {node.name}', 'magenta')
      cprint(f'class node docstring: {ast.get_docstring(node)}', 'magenta')
      cprint(f'class node source code: {ast.get_source_segment(source_code, node)}', 'magenta')
      cprint('-' * 80)

In [41]:
%%time
walk_python_code('demo_python_source.py')

func node name: example_func_0
func node docstring: None
func node source code: def example_func_0():
  print("Hello")
--------------------------------------------------------------------------------
func node name: example_func_1
func node docstring: None
func node source code: def example_func_1(arg0, arg1):
  print(arg0, arg1)
--------------------------------------------------------------------------------
node <ast.FunctionDef object at 0x1062519f0> already has a docstring
--------------------------------------------------------------------------------
class node name: ExampleClass
class node docstring: None
class node source code: class ExampleClass:
  def __init__(self, value) -> None:
    self.value = value
  
  def print(self):
    print(self.value)

  def print_reversed(self):
    print(reversed(self.value))
--------------------------------------------------------------------------------
func node name: __init__
func node docstring: None
func node source code: def __init__(sel